# Chromium

In [ ]:
import subprocess
import json
import os
import datetime

In [ ]:
GIT_REPO = '/root/hstsadoption.github.io'

In [ ]:
def getChromiumPreloadSize():
    """Loads the chromium preload list and counts the hsts entries"""
    
    count = subprocess.run(['./chromium-count.sh'], check=True, stdout=subprocess.PIPE).stdout.decode('utf-8')
    return int(count)

In [ ]:
# Load preload list size
print("Analyzing chromium preload list")
timestamp = int(datetime.datetime.now().timestamp()) * 1000
size = getChromiumPreloadSize()

In [ ]:
# Update repository
print("Updating local repository")
subprocess.run(['git', '-C', GIT_REPO, 'pull'])

In [ ]:
# Update data files
print("Updating data files")
for ds in ['ipv4', 'ipv6', 'merged']:
    file = os.path.join(GIT_REPO, 'data', 'configuration', 'absolute', ds + '.json')
    configData = {}
    
    with open(file, 'r') as f:
        configData = json.loads(f.read())
        
    configData['chromium'] = configData.get('chromium', [])
    configData['chromium'].append([timestamp, size])
    configData['chromium'].sort(key=lambda x: x[0])
    
    with open(file, 'w') as f:
        json.dump(configData, f, indent="\t", sort_keys=True)
    
    # Add file to git
    subprocess.run(['git', '-C', GIT_REPO, 'add', file])

In [ ]:
# Deploy changes
print("Deploying changes")
subprocess.run(['git', '-C', GIT_REPO, 'commit', '-m', "automated update (chromium preload list)"])
subprocess.run(['git', '-C', GIT_REPO, 'push'])